In this paper, we will attempt to figure out the structure of a marine food web of the Carribean Sea. The data come courtesy of web of life,http://www.web-of-life.es/map.php?type=6 an ecological network database, and has been used in the 2005 paper Interaction strength combinations and the overfishing of a marine food web by Bascompte et al. https://www.pnas.org/content/102/15/5443
The network summarize the trophic interactions of the members of a tropical marine ecosystem, including detritus, fishes, sharks, and producers such as benthic algea and phytoplankton: the members are vertices and the trophic interactions are summarized by weighted edges, the weights being the strength of the trophic interaction between a prey and its predator, estimated by measuring the proportion of prey biomass consumed per capita (per unit of biomass of the predator), per day. The predator is the target of this interaction and the prey, its source. The network is thus a directed, weighted graph, composed of 249 nodes and 3313 edges.
Because we are primarily interested in the structure of the food web, we began by running a community detection algorithm, using the Gephy implementation which conveniently works on directed, weighted graphs.

![Figure 1](https://i.imgur.com/IHrccIt.jpg)
Figure 1: Visualisation of the food web and its communities. Node colors represents the seven communities that were identified, node size represents the weighted out-degree and edge size, the strength of the trophic interaction. The network layout has been computed using Force Atlas. Preys are the source of the edges, and predators the target.

In Figure 1 we can see that seven communities has been observed. They are organized around one or more preys or prey groups (e.g. Gastropods, Microfauna, Shrimps...), which are the main food sources for the predator species of the community. Those key prey groups and species can be identified with their high weighted out-degree, which indicates that they are major sources of alimentation for many species.
With a modularity of 0.447, the identified communities can explain a full half of the interactions.
Indeed, if we contract and simplify the network by merging every nodes into its community, we observe that the large majorities of interactions are intra-communities:
![Figure 2](https://i.imgur.com/Yab5f3i.png)
Figure 2: Predation is mostly intra-community. Simplified graph depicting the identified communities and their interactions. Node size is weighted degree, edge width is interaction strength. Communities are named for their central prey species.

Thus, although the food web is complex, it does have an underlying structure of communities, each composed of specialist species that feed on one particular type of prey almost exclusively. That makes it interesting to see which preys are grouped together: crabs and shrimps, for example, are both in the same community, indicating that there is significant overlap between the predators of both groups. Benthic autotrophs (ie sea-floor algea) and Detritus POM DOM (ie, dead organic matter) are also grouped, illustrating that bottom feeders feeding on dead matter on the sea floor often feed on benthic algea as well.

The communities are not completely isolated though: notably, there are species feeding not directly on the prey groups at the center of the communities, but rather on the various predators of the communities. Those super-predators can easily be identified on the graph:
![Figure 3](https://imgur.com/a/PETu5L1)
Figure 3: Apex predators of the Carribean marine biome. Node size is in-degree, illustrating the broad spectrum of species  apex predators can consume. The graph layout is Force Atlas.

On figure 3,  apex predators such as Galeocerdo cuvieri (the tiger shark) or Carcharhinus perezi, the Carribean reef shark have been pushed to the periphery of the graph by the Force Atlas algorithm, because of the many but weak trophic ties with its prey species: apex predators consume very diverse species, and do not focus on any particular one, so the individual trophic interactions are weak. 
![Figure 4](https://i.imgur.com/7Xr5d6P.jpg)
Figure 4: Apex predators have high closeness. Node size is proportional to closeness centrality. The algorithm used is undirected and unweighted.

On figure 4, closeness centrality is evaluted for the species of the food web. Apex predator species have relatively higher closeness, reflecting the large number of species they can directly affect. This give them a disproportionate impact on their ecosystem: a variation in their population size will have a strong effect on every member of the food web as they are close to every member. 
![Figure 5](https://i.imgur.com/vJguYmx.jpg)
Figure 5: Repartition of betweenness centrality. Galeocerdo cuvieri and Aetobatus narinari are the tiger shark and the spotted eagle ray, respectively. Node size is proportional to betweeness centrality. The three communities are centred on bottom feeders or benthivore (green), pelagic species (orange) and benthic species (violet).

In conclusion, the food wed can be simplified for modelization purposes in the following way:
![Figure 6](https://i.imgur.com/0qOopQh.png)
This could help to better understand the consequences of overfishing on the larger ecosystem.

Most of this was realized in igraph then exported out to gephy for visualization. See below for code annexes.

In [1]:
import pandas as pd
file = pd.read_excel(r'C:\Users\solal\Desktop\web-of-life_2019-12-07_155048\FW_008.xls')
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
file=np.matrix(file)
G = nx.DiGraph()
for i in range(1,len(file)):
    for j in range(1,len(file)):
        if file[i,j]!=0.:
            G.add_edge(file[i,0], file[j,0],weight=file[i,j])
import igraph as ig
nx.write_graphml(G,'graph.graphml') # Export NX graph to file
Gix = ig.read('graph.graphml',format="graphml") # Create new IG graph from file

In [11]:
Gix = ig.read('C:/Users/solal/Documents/Untitled.graphml',format="graphml") # Create new IG graph from file
L=[]
for i in Gix.vs:#to combine the graph into communities
    if i['Modularity Class']==0:
        i['label']="Microfauna community"
        L.append(0)
    if i['Modularity Class']==1:
        L.append(1)
        i['label']="Gastropods, Bivalvs, Echinoids, Isopods community"
    if i['Modularity Class']==2:
        L.append(2)
        i['label']="Crabs, Shrimps community"
    if i['Modularity Class']==3:
        L.append(3)
        i['label']="Benthic autotrophs, detritus community"
    if i['Modularity Class']==4:
        L.append(4)
        i['label']="Sponges, Symbiotic algea, Sea anemones community"
    if i['Modularity Class']==5:
        L.append(5)
        i['label']="Phytoplankton, Hyporhamphus unifasciatus communityy"
    if i['Modularity Class']==6:
        L.append(6)
        i['label']="Polychaetes community"
Gix.contract_vertices(L,combine_attrs={'label':'first'})
del(Gix.es['Edge Label'])
del(Gix.es['id'])
Gix.simplify(loops=False,combine_edges=sum)
ig.plot(Gix)
Gix.write_graphml('C:/Users/solal/Documents/simplifiedGraph.graphml')

[3, 0, 4, 4, 4, 6, 4, 5, 3, 1, 1, 6, 3, 3, 1, 2, 1, 1, 2, 5, 3, 2, 3, 4, 3, 3, 3, 5, 5, 3, 4, 2, 0, 3, 3, 3, 2, 3, 3, 6, 3, 2, 3, 3, 3, 2, 3, 0, 6, 3, 3, 3, 4, 3, 1, 1, 4, 6, 3, 4, 3, 4, 3, 1, 4, 0, 4, 3, 0, 3, 4, 4, 3, 3, 2, 2, 3, 2, 4, 4, 3, 6, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 6, 6, 5, 0, 0, 0, 2, 2, 2, 0, 2, 0, 0, 0, 0, 2, 0, 0, 6, 6, 0, 0, 0, 0, 2, 3, 0, 0, 5, 2, 0, 2, 6, 0, 0, 0, 2, 2, 3, 2, 5, 0, 2, 6, 0, 2, 0, 0, 5, 0, 1, 0, 0, 2, 0, 5, 0, 0, 6, 1, 0, 0, 0, 0, 0, 1, 2, 1, 1, 2, 2, 2, 6, 2, 2, 2, 2, 6, 1, 2, 2, 2, 1, 2, 2, 1, 1, 2, 1, 1, 1, 2, 2, 2, 1, 2, 2, 5, 2, 1, 5, 1, 2, 2, 2, 0, 0, 0, 0, 0, 0, 2, 5, 2, 0, 3, 5, 2, 0, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 5, 5, 1, 1, 0, 2, 3, 2, 0, 0, 0, 3, 5, 6, 6, 5, 0]


c:\users\solal\appdata\local\programs\python\python37\lib\site-packages\igraph\__init__.py:2223: RuntimeWarning: Could not add vertex ids, there is already an 'id' vertex attribute at d:\build\igraph\igraph-vtraag\igraph-0.7.1+5.3b99dbf6-msvc\src\foreign-graphml.c:443
  return reader(f, *args, **kwds)


'IGRAPH D-W- 7 48 -- \n+ attr: label (v), weight (e)'

In [2]:
import igraph as ig
Gix = ig.read('C:/Users/solal/Documents/Untitled.graphml',format="graphml")#recap of the foodchain
L=[]
namelist=['Carcharhinus perezi','Galeocerdo cuvieri','Carcharhinus falciformis','Ginglymostoma cirratum','Mustelus canis','Aetobatus narinari','Negaprion brevirostris','Sphyrna tiburo','Rhizoprionodon porosus','Carcharhinus acronotus']
for j in range(len(Gix.vs)):
    i=Gix.vs[j]
    if i['label']=='Microfauna':
        L.append(7)
    if i['label']=='Benthic autotrophs':
        L.append(8)
    if i['label']=='Crabs':
        L.append(9)
    if i['label']=='Shrimps':
        L.append(10)
    if i['label']=='Detritus. POM. DOM':
        L.append(11)
    if i['label']=='Polychaetes':
        L.append(12)
    if i['label']=='Gastropods':
        L.append(13)
    if i['label']=='Bivalves':
        L.append(14)
    if i['label']=='Sponges':
        L.append(15)
    if i['label']=='Echinoids':
        L.append(16)
    if i['label']=='Hyporamphus unifasciatus':
        L.append(17)
    if i['label']=='Phytoplankton':
        L.append(18)
    if i['label']=='Zooplankton':
        L.append(19)
    if i['label']=='Isopods':
        L.append(20)
    if i['label']=='Symbiotic algea':
        L.append(21)
    #if i['label'] in namelist:#for some reasons, if this category is included, Gix.simplify will crash python
        #i['label']='Superpredators'
        #L.append(22)
    if i['Modularity Class']==0 and j==len(L):
        i['label']="Microfauna community"
        L.append(0)
    if i['Modularity Class']==1 and j==len(L):
        L.append(1)
        i['label']="Gastropods, Bivalvs, Echinoids, Isopods community"
    if i['Modularity Class']==2 and j==len(L):
        L.append(2)
        i['label']="Crabs, Shrimps community"
    if i['Modularity Class']==3 and j==len(L):
        L.append(3)
        i['label']="Benthic autotrophs, detritus community"
    if i['Modularity Class']==4 and j==len(L):
        L.append(4)
        i['label']="Sponges, Symbiotic algea, Sea anemones community"
    if i['Modularity Class']==5 and j==len(L):
        L.append(5)
        i['label']="Phytoplankton, Hyporhamphus unifasciatus communityy"
    if i['Modularity Class']==6 and j==len(L):
        L.append(6)
        i['label']="Polychaetes community"
Gix.contract_vertices(L,combine_attrs={'label':'random'})
del(Gix.es['Edge Label'])
del(Gix.es['id'])
Gix.simplify(loops=False,combine_edges=dict(weight=sum))
ig.plot(Gix)
Gix.write_graphml('C:/Users/solal/Documents/simplifiedGraph2.graphml')

ModuleNotFoundError: No module named 'igraph'

In [153]:
import igraph as ig
Gix = ig.read('C:/Users/solal/Documents/Untitled.graphml',format="graphml")
Gix.vs['Weighted Directed Betweenness']=Gix.betweenness(weights='weight',directed=True)#gephy doesn't have this for some reasons
Gix.vs['Weighted Undirected Betweenness']=Gix.betweenness(weights='weight',directed=False)
Gix.write_graphml('C:/Users/solal/Documents/Eccentricity.graphml')

IGRAPH D-W- 249 3313 -- 
+ attr: Betweenness Centrality (v), Closeness Centrality (v), Component ID (v), Harmonic Closeness Centrality (v), In-Degree (v), Modularity Class (v), Out-Degree (v), PageRank (v), Strongly-Connected ID (v), Weighted Degree (v), Weighted Directed Betweenness (v), Weighted Directed Closeness (v), Weighted In-Degree (v), Weighted Out-Degree (v), Weighted Undirected Betweenness (v), Weighted Undirected Closeness (v), b (v), g (v), id (v), label (v), r (v), size (v), x (v), y (v), Edge Label (e), id (e), weight (e)


In [3]:
import igraph as ig
Gix = ig.read('C:/Users/solal/Documents/Untitled.graphml',format="graphml")
i=0
while Gix.vs[i]['label']!='Galeocerdo cuvieri':
    i+=1
SP=Gix.shortest_paths_dijkstra(source=Gix.vs[i],mode='IN')
Gix.vs['Out-Degree']=[SP[0][j] for j in range(len(SP[0]))]
Gix.write_graphml('C:/Users/solal/Documents/Untitled.graphml')

ModuleNotFoundError: No module named 'igraph'

In [145]:
import igraph as ig
Gix = ig.read('C:/Users/solal/Documents/Untitled.graphml',format="graphml")#we want to identify the ratio of edges in/out of community
print(Gix.summary())
Gix.vs['CommunityRatioIn']=list(range(len(Gix.vs)))
Gix.vs['CommunityRatioOut']=list(range(len(Gix.vs)))
Gix.vs['CommunityRatio']=list(range(len(Gix.vs)))
Gix.vs['ModularityClass']=Gix.vs['Modularity Class']
for i in range(len(Gix.vs)):
    ModClass=Gix.vs[i]['Modularity Class']
    EdgeListOut=Gix.es.select(_source=i)#all edges from i
    EdgeListIn=Gix.es.select(_target=i)#all edges to i
    if len(EdgeListIn)==0:
        Gix.vs[i]['CommunityRatioIn']=0
        target_vertex = Gix.vs[[j.target for j in EdgeListOut]]#we select the vertices that are targets of edges from i
        vOut=target_vertex.select(ModularityClass_eq=ModClass)#among those, we select those of same community as i
        wOut=Gix.es.select(_target_in=wOut,_source=i)#we then select the edges between those vertices and i
        Gix.vs[i]['CommunityRatioOut']=sum(wOut['weight'])/sum(EdgeListOut['weight'])#we can then make a weighted ratio
        Gix.vs[i]['CommunityRatio']=sum(wOut['weight'])/sum(EdgeListOut['weight'])#if EdgeListIn is empty, total ratio is equal to out ratio
    else:
        source_vertex = Gix.vs[[j.source for j in EdgeListIn]]#select all vertices that are sources of edges to i
        vIn=source_vertex.select(ModularityClass_eq=ModClass)
        wIn=Gix.es.select(_source_in=vIn,_target=i)
        Gix.vs[i]['CommunityRatioIn']=sum(wIn['weight'])/sum(EdgeListIn['weight'])
        if len(EdgeListOut)==0:
            Gix.vs[i]['CommunityRatioOut']=0
            Gix.vs[i]['CommunityRatio']=sum(wIn['weight'])/sum(EdgeListIn['weight'])
        else:
            target_vertex = Gix.vs[[j.target for j in EdgeListOut]]
            vOut=target_vertex.select(ModularityClass_eq=ModClass)
            wOut=Gix.es.select(_target_in=vOut,_source=i)
            Gix.vs[i]['CommunityRatioOut']=sum(wOut['weight'])/sum(EdgeListOut['weight'])
            Gix.vs[i]['CommunityRatio']=(sum(wOut['weight'])+sum(wIn['weight'])/sum(EdgeListIn['weight']))/(sum(EdgeListIn['weight'])+sum(EdgeListOut['weight']))
Gix.write_graphml('C:/Users/solal/Documents/CommunityRatio.graphml')

IGRAPH D-W- 249 3313 -- 
+ attr: Betweenness Centrality (v), Closeness Centrality (v), Component ID (v), Harmonic Closeness Centrality (v), In-Degree (v), Modularity Class (v), Out-Degree (v), PageRank (v), Strongly-Connected ID (v), Weighted Degree (v), Weighted In-Degree (v), Weighted Out-Degree (v), b (v), g (v), id (v), label (v), r (v), size (v), x (v), y (v), Edge Label (e), id (e), weight (e)
